In [2]:
import os
import shutil


image_dir = r'/home/kcdh/Desktop/madhuri/crop_six class_images'
label_dir = r'/home/kcdh/Desktop/madhuri/crop_six class_labels'
output_dir = r'/home/kcdh/Desktop/madhuri/output'


os.makedirs(output_dir, exist_ok=True)

for i in range(6):
    os.makedirs(os.path.join(output_dir, f'class_{i}'), exist_ok=True)

# Read text files and move images
for txt_file in os.listdir(label_dir):
    with open(os.path.join(label_dir, txt_file), 'r') as file:
        first_line = file.readline().strip()
        class_label = int(first_line.split()[0])  
    
    image_name = txt_file.replace('.txt', '.jpg')  
    image_path = os.path.join(image_dir, image_name)
    if os.path.exists(image_path):
        shutil.move(image_path, os.path.join(output_dir, f'class_{class_label}', image_name))
    else:
        print(f"Image {image_name} not found.")

In [3]:
import sklearn

In [2]:
import os
import shutil

from sklearn.model_selection import train_test_split

# Define your directories
base_dir = r'/home/kcdh/Desktop/madhuri/output'  # Update this to your base dataset folder
output_dir = r'/home/kcdh/Desktop/madhuri/cross_val_output'  # Folder to store cross-validation splits

# Organize images by patient number
patient_images = {}
for class_folder in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_folder)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            patient_number = image_name.split('_')[0]
            image_path = os.path.join(class_path, image_name)
            if patient_number not in patient_images:
                patient_images[patient_number] = []
            patient_images[patient_number].append((image_path, class_folder))

# Split patient numbers into 3 sets (for 3-fold cross-validation)
patient_numbers = list(patient_images.keys())
train_patients = [p for p in patient_numbers if p.startswith('00')]
remaining_patients = [p for p in patient_numbers if not p.startswith('00')]

# Create 3-fold cross-validation sets (70% train, 30% test)
for fold in range(3):
    train_patients = [p for p in patient_numbers if p.startswith('00')]
    remaining_patients = [p for p in patient_numbers if not p.startswith('00')]
    
    train_patients_fold, test_patients_fold = train_test_split(remaining_patients, test_size=0.2, random_state=fold)
    train_patients_fold.extend(train_patients)

    # Define train and test directories for each fold
    train_dir = os.path.join(output_dir, f'fold_{fold+1}', 'train')
    test_dir = os.path.join(output_dir, f'fold_{fold+1}', 'test')

    # Create directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Move train images
    for patient in train_patients_fold:
        for image_path, class_folder in patient_images[patient]:
            dest_folder = os.path.join(train_dir, class_folder)
            os.makedirs(dest_folder, exist_ok=True)
            shutil.copy(image_path, dest_folder)  # Using copy to avoid file deletion

    # Move test images
    for patient in test_patients_fold:
        for image_path, class_folder in patient_images[patient]:
            dest_folder = os.path.join(test_dir, class_folder)
            os.makedirs(dest_folder, exist_ok=True)
            shutil.copy(image_path, dest_folder)  # Using copy to avoid file deletion

In [5]:
import os
def counting_img(parent_fol):
    
    for root,dir,files in os.walk(parent_fol):
        num_imgs  = 0
        for file in files:
            if file.endswith('.jpg'):
                num_imgs+=1
        print(f"{root}: {num_imgs} images")
                
parent_dir = r'/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train'
counting_img(parent_dir)

/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train: 0 images
/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train/class_0: 27 images
/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train/class_1: 47 images
/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train/class_5: 203 images
/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train/class_3: 323 images
/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train/class_4: 93 images
/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train/class_2: 43 images


In [2]:
import torchvision.models as models
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

#  function to load images from a directory and apply transformations
def load_images_from_folder(folder, transform=None):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))
    
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = Image.open(image_path).convert('RGB')
            if transform:
                image = transform(image)
            images.append(image)
            labels.append(label)
    
    return images, labels


parent_dir = r'/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/train'
train_images, train_labels = load_images_from_folder(parent_dir, transform=transform)

# Convert to PyTorch tensors and create a DataLoader
train_images = torch.stack(train_images)
train_labels = torch.tensor(train_labels)
train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)





#  ResNet18 model for feature extraction
class ResNet18EmbeddingModel(nn.Module):
    def _init_(self):
        super(ResNet18EmbeddingModel, self)._init_()
    
        resnet18 = models.resnet18(weights = ResNet18_weights.DEFAULT)
        
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(resnet18.children())[:-1])
    
    def forward(self, x):
        
        x = self.features(x)
        x = torch.flatten(x, 1)  
        return x


model = ResNet18EmbeddingModel()


for images, labels in train_loader:
    features = model(images)
    print(f"Extracted features shape: {features.shape}")

AttributeError: 'ResNet18EmbeddingModel' object has no attribute 'features'

In [ ]:
model.train()


criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)


for epoch in range(10):  
    for images, labels in train_loader:
    
        embeddings = model(images)
        

        loss = criterion(embeddings, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

# Save the trained embedding model
torch.save(model.state_dict(), 'resnet18_embedding_model.pth')

In [ ]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Function to load images and apply transformations
def load_images_from_folder(folder, transform=None):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))
    
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = Image.open(image_path).convert('RGB')
            if transform:
                image = transform(image)
            images.append(image)
            labels.append(label)
    
    return images, labels

#  test images
test_dir = r'/home/kcdh/Desktop/madhuri/cross_val_output/fold_1/test'
test_images, test_labels = load_images_from_folder(test_dir, transform=transform)



test_images, test_labels = load_images_from_folder(test_dir, transform=transform)
test_images = torch.stack(test_images)
test_labels = torch.tensor(test_labels)


model.eval()  


with torch.no_grad():
    test_features = model(test_images)  # Features from ResNet18


    
num_classes = 6  
input_dim = test_features.size(1)  # Number of features from ResNet18 model

classifier = nn.Linear(input_dim, num_classes)

# Evaluate the classifier
classifier.eval()
with torch.no_grad():
    test_logits = classifier(test_features)
    test_predictions = torch.argmax(F.softmax(test_logits, dim=1), dim=1)

# Calculate accuracy
accuracy = (test_predictions == test_labels).float().mean().item()
print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [ ]:
[8:38 PM, 9/11/2024] Madhuri: transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

# Function to load images and apply transformations
def load_images_from_folder(folder, transform=None):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))
    
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = Image.open(image_path).convert('RGB')
            if transform:
                image = transform(image)
            images.append(image)
            labels.append(label)
    
    return images, labels

# Load test images
test_dir = r'C:\Users\91939\OneDrive\Desktop\lesion3\test'
test_images, test_labels = load_images_from_folder(test_dir, transform=transform)



test_images, test_labels = load_images_from_folder(test_dir, transform=transform)
test_images = torch.stack(test_images)
test_labels = torch.tensor(test_labels)

# Feature extraction using the pre-trained ResNet18 model
model.eval() 





import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score



with torch.no_grad():
    test_features = model(test_images)  # Features from ResNet18

# Convert features to numpy
test_features = test_features.cpu().numpy()
test_labels = test_labels.cpu().numpy()

# Define the parameter grid for the DecisionTreeClassifier
param_grid = {
    'criterion': ['gini', 'entropy'],  # Measure for splitting: gini or entropy
    'max_depth': [None, 10, 20, 30, 40],  # Max depth of the tree
    'min_samples_split': [2, 4, 6, 8],  # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples in a leaf node
    'max_features': [None, 'sqrt', 'log2']  # Number of features to consider for best split
}


decision_tree = DecisionTreeClassifier()


grid_search = GridSearchCV(decision_tree, param_grid, cv=3, n_jobs=-1, verbose=2)


grid_search.fit(test_features, test_labels)

best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

print("Best Parameters:", best_params)


dt_predictions = best_model.predict(test_features)
dt_accuracy = accuracy_score(test_labels, dt_predictions)

print(f"Decision Tree Test Accuracy with Best Parameters: {dt_accuracy * 100:.2f}%")


print("Decision Tree Classification Report:")
print(classification_report(test_labels, dt_predictions))


confusion_mat = confusion_matrix(test_labels, dt_predictions)
print("Confusion Matrix:")
print(confusion_mat)

class_accuracies = confusion_mat.diagonal() / confusion_mat.sum(axis=1)

# Print accuracies for each class
for i, accuracy in enumerate(class_accuracies):
    print(f"Accuracy for class {i}: {accuracy * 100:.2f}%")


In [ ]:
with torch.no_grad():
    test_features = model(test_images)  # Features from ResNet18

    

param_grid = {
    'n_estimators': [100, 200, 300],  
    'criterion': ['gini', 'entropy'],  
    'max_depth': [ None,10, 20, 30],  
    'min_samples_split': [2, 4, 6], 
    'min_samples_leaf': [1, 2, 4],  
    'max_features': ['sqrt', 'log2',None],  
    'bootstrap': [True, False]  # Whether to use bootstrap samples
}


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
import torch
import numpy as np

# Assuming your test features and labels have already been extracted
# and stored in test_features and test_labels

# Define the Random Forest classifier
rf = RandomForestClassifier()

# Perform grid search with 5-fold cross-validation
grid_search = GridSearchCV(rf, param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(test_features, test_labels)

# Get the best parameters found by grid search
print("Best Parameters:", grid_search.best_params_)

# Evaluate the best model on test data
best_rf = grid_search.best_estimator_
rf_predictions = best_rf.predict(test_features)

rf_accuracy = accuracy_score(test_labels, rf_predictions)

print(f"Random Forest Test Accuracy with Best Parameters: {rf_accuracy * 100:.2f}%")

from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

print("Random Forest Classification Report:")
print(classification_report(test_labels, rf_predictions))


confusion_mat = confusion_matrix(test_labels, rf_predictions)
print("Confusion Matrix:")
print(confusion_mat)

class_accuracies = confusion_mat.diagonal() / confusion_mat.sum(axis=1)

# Print accuracies for each class
for i, accuracy in enumerate(class_accuracies):
    print(f"Accuracy for class {i}: {accuracy * 100:.2f}%")

In [ ]:
#################25/9/24

In [4]:
import os
import shutil
from sklearn.model_selection import KFold

# Define your directories
base_dir = r'/home/kcdh/Desktop/madhuri/output'  # Update this to your base dataset folder
output_dir = r'/home/kcdh/Desktop/madhuri/cross_val' # Folder to store cross-validation splits

# Organize images by patient number
patient_images = {}
for class_folder in os.listdir(base_dir):
    class_path = os.path.join(base_dir, class_folder)
    if os.path.isdir(class_path):
        for image_name in os.listdir(class_path):
            patient_number = image_name.split('_')[0]
            image_path = os.path.join(class_path, image_name)
            if patient_number not in patient_images:
                patient_images[patient_number] = []
            patient_images[patient_number].append((image_path, class_folder))

# Split patient numbers into train (with '00') and remaining patients
train_patients = [p for p in patient_images if p.startswith('00')]
remaining_patients = [p for p in patient_images if not p.startswith('00')]

# Use KFold for the remaining patients (3-fold cross-validation)
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# Generate fold splits
for fold, (_, test_idx) in enumerate(kf.split(remaining_patients)):
    # Get the corresponding patients for the test set
    test_patients_fold = [remaining_patients[i] for i in test_idx]
    
    # Combine train_patients (with '00') with remaining patients for the train set
    train_patients_fold = train_patients + [remaining_patients[i] for i in range(len(remaining_patients)) if i not in test_idx]

    # Define train and test directories for each fold
    train_dir = os.path.join(output_dir, f'fold_{fold+1}', 'train')
    test_dir = os.path.join(output_dir, f'fold_{fold+1}', 'test')

    # Create directories if they don't exist
    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Move train images
    for patient in train_patients_fold:
        for image_path, class_folder in patient_images[patient]:
            dest_folder = os.path.join(train_dir, class_folder)
            os.makedirs(dest_folder, exist_ok=True)
            shutil.copy(image_path, dest_folder)  # Copying to avoid file deletion

    # Move test images
    for patient in test_patients_fold:
        for image_path, class_folder in patient_images[patient]:
            dest_folder = os.path.join(test_dir, class_folder)
            os.makedirs(dest_folder, exist_ok=True)
            shutil.copy(image_path, dest_folder)  # Copying to avoid file deletion

In [5]:
import os
def counting_img(parent_fol):
    
    for root,dir,files in os.walk(parent_fol):
        num_imgs  = 0
        for file in files:
            if file.endswith('.jpg'):
                num_imgs+=1
        print(f"{root}: {num_imgs} images")
                
parent_dir = r'/home/kcdh/Desktop/madhuri/cross_val/fold_1'
counting_img(parent_dir)

/home/kcdh/Desktop/madhuri/cross_val/fold_1: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/train: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/train/class_0: 26 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/train/class_1: 41 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/train/class_5: 187 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/train/class_3: 279 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/train/class_4: 69 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/train/class_2: 36 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/test: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/test/class_0: 3 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/test/class_1: 20 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/test/class_5: 70 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/test/class_3: 103 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/test/class_4: 28 images
/home/kcdh/Desktop/madhuri/cross_val/fold_1/test/class_2: 12 images


In [7]:
import os
def counting_img(parent_fol):
    
    for root,dir,files in os.walk(parent_fol):
        num_imgs  = 0
        for file in files:
            if file.endswith('.jpg'):
                num_imgs+=1
        print(f"{root}: {num_imgs} images")
                
parent_dir = r'/home/kcdh/Desktop/madhuri/cross_val/fold_2'
counting_img(parent_dir)

/home/kcdh/Desktop/madhuri/cross_val/fold_2: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/train: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/train/class_0: 23 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/train/class_1: 57 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/train/class_5: 157 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/train/class_3: 279 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/train/class_4: 64 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/train/class_2: 37 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/test: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/test/class_0: 6 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/test/class_1: 4 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/test/class_5: 100 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/test/class_3: 103 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/test/class_4: 33 images
/home/kcdh/Desktop/madhuri/cross_val/fold_2/test/class_2: 11 images


In [8]:
import os
def counting_img(parent_fol):
    
    for root,dir,files in os.walk(parent_fol):
        num_imgs  = 0
        for file in files:
            if file.endswith('.jpg'):
                num_imgs+=1
        print(f"{root}: {num_imgs} images")
                
parent_dir = r'/home/kcdh/Desktop/madhuri/cross_val/fold_3'
counting_img(parent_dir)

/home/kcdh/Desktop/madhuri/cross_val/fold_3: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/train: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/train/class_0: 16 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/train/class_1: 38 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/train/class_5: 195 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/train/class_3: 265 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/train/class_4: 85 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/train/class_2: 30 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/test: 0 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/test/class_0: 13 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/test/class_1: 23 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/test/class_5: 62 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/test/class_3: 117 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/test/class_4: 12 images
/home/kcdh/Desktop/madhuri/cross_val/fold_3/test/class_2: 18 images


In [10]:
import torchvision.models as models
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import os
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F

# Define the transformations
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

#  function to load images from a directory and apply transformations
def load_images_from_folder(folder, transform=None):
    images = []
    labels = []
    class_names = sorted(os.listdir(folder))
    
    for label, class_name in enumerate(class_names):
        class_path = os.path.join(folder, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            image = Image.open(image_path).convert('RGB')
            if transform:
                image = transform(image)
            images.append(image)
            labels.append(label)
    
    return images, labels


parent_dir = r'/home/kcdh/Desktop/madhuri/cross_val/fold_1/train'
train_images, train_labels = load_images_from_folder(parent_dir, transform=transform)

# Convert to PyTorch tensors and create a DataLoader
train_images = torch.stack(train_images)
train_labels = torch.tensor(train_labels)
train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)





# #  ResNet18 model for feature extraction
# class ResNet18EmbeddingModel(nn.Module):
#     def _init_(self):
#         super(ResNet18EmbeddingModel, self)._init_()
    
#         resnet18 = models.resnet18(weights = ResNet18_weights.DEFAULT)
        
#         # Remove the final fully connected layer
#         self.features = nn.Sequential(*list(resnet18.children())[:-1])
    
#     def forward(self, x):
        
#         x = self.features(x)
#         x = torch.flatten(x, 1)  
#         return x


# model = ResNet18EmbeddingModel()


# for images, labels in train_loader:
#     features = model(images)
#     print(f"Extracted features shape: {features.shape}")

In [11]:

#  ResNet18 model for feature extraction
class ResNet18EmbeddingModel(nn.Module):
    def _init_(self):
        super(ResNet18EmbeddingModel, self)._init_()
    
        resnet18 = models.resnet18(weights = models.ResNet18_Weights.IMAGENET1K_V1)
        
        # Remove the final fully connected layer
        self.features = nn.Sequential(*list(resnet18.children())[:-1])
    
    def forward(self, x):
        
        x = self.features(x)
        x = torch.flatten(x, 1)  
        return x


model = ResNet18EmbeddingModel()


In [12]:
for images, labels in train_loader:
    features = model(images)
    print(f"Extracted features shape: {features.shape}")

AttributeError: 'ResNet18EmbeddingModel' object has no attribute 'features'